In [4]:
%pip install python-dotenv --upgrade --quiet langchain langchain-groq groq

In [5]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser

router_llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.0)

expert_llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.7)

MODEL_CONFIG = {
    "technical": "You are a senior technical support engineer. You are rigorous, precise, and focus heavily on code, debugging, and system architecture. Provide direct technical solutions without fluff.",
    "billing": "You are a billing support specialist. You are empathetic, financially focused, and policy-driven. Help the user understand charges, refunds, and subscription tiers cleanly.",
    "general": "You are a helpful general customer support agent. Answer casual queries in a friendly and concise manner.",
    "tool_use": "You are a data routing specialist. The user needs real-time fetched data." # For the bonus challenge
}

In [9]:
def route_prompt(user_input):
    routing_template = ChatPromptTemplate.from_messages([
        ("system", "Classify the following user input into exactly one of these categories: [technical, billing, tool_use, general]. Return ONLY the category name in lowercase. Do not include punctuation, reasoning, or extra words."),
        ("human", "{input}")
    ])

    chain = routing_template | router_llm | StrOutputParser()

    category = chain.invoke({"input": user_input}).strip().lower()

    valid_categories = ["technical", "billing", "tool_use", "general"]
    if category not in valid_categories:
        return "general"

    return category

print("Router Test 1:", route_prompt("My python script is throwing an IndexError on line 5."))
print("Router Test 2:", route_prompt("I was charged twice for my subscription this month."))
print("Router Test 3:", route_prompt("What is the current price of Bitcoin?"))

Router Test 1: technical
Router Test 2: billing
Router Test 3: billing


In [10]:
# Mock function for the Bonus Challenge
def mock_get_bitcoin_price():
    return "$65,432.10"

def process_request(user_input):
    # 1. Route the input
    category = route_prompt(user_input)
    print(f"--> [System] Request routed to: '{category.upper()}' expert")

    # 2. Bonus: Tool Use execution
    if category == "tool_use":
        if "bitcoin" in user_input.lower() or "btc" in user_input.lower():
            price = mock_get_bitcoin_price()
            return f"Tool Output: The current fetched price of Bitcoin is {price}."
        else:
            return "Tool Output: Mock tool data requested but no API exists for this specific query."

    # 3. Select the correct System Prompt for standard categories
    system_prompt = MODEL_CONFIG[category]

    expert_template = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{input}")
    ])

    # 4. Call the LLM with the specific expert config
    chain = expert_template | expert_llm | StrOutputParser()

    return chain.invoke({"input": user_input})

In [11]:
test_queries = [
    "My python script is throwing an IndexError on line 5.",
    "I was charged twice for my subscription this month. I need a refund.",
    "Hey, how is your day going?",
    "What is the current price of Bitcoin right now?"
]

for i, query in enumerate(test_queries):
    print(f"\n--- Test {i+1} ---")
    print(f"User: {query}")
    response = process_request(query)
    print(f"\nAgent:\n{response}")
    print("-" * 40)


--- Test 1 ---
User: My python script is throwing an IndexError on line 5.
--> [System] Request routed to: 'TECHNICAL' expert

Agent:
To troubleshoot the issue, I'll need more information. Please provide:

1. The relevant part of the code that's throwing the error (line 5 and surrounding lines).
2. The exact error message (including any line numbers or stack traces).
3. Any relevant variables or data structures involved in the error.

With this information, I can help you identify the cause of the IndexError and provide a solution.

If you're not comfortable sharing the code, a general example of an IndexError in Python looks like this:

```python
my_list = [1, 2, 3]
print(my_list[3])  # IndexError: list index out of range
```

In this case, the error occurs because the list `my_list` only has 3 elements, and we're trying to access the 4th element (index 3).
----------------------------------------

--- Test 2 ---
User: I was charged twice for my subscription this month. I need a refu